In [1]:
#pip install llama-index


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.9 MB 5.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.4/11.9 MB 5.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/11.9 MB 5.8 MB/s eta 0:00:02
   -------------- ------------------------- 4.5/11.9 MB 5.5 MB/s eta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [3]:
#pip install llama-index qdrant_client torch transformers


  Using cached qdrant_client-1.15.1-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached portalocker-3.2.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached h2-4.3.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached platformdirs-4.5.0-py3-none-any.whl.metadata (12 kB)
Using cached qdrant_client-1.15.1-py3-none-any.whl (337 kB)
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.8/109.3 MB 3.7 MB/s eta 0:00:29
    --------------------------------------- 1.8/109.3 MB 4.6 MB/s eta 0:00:24
   - ----------------------

In [4]:
#pip install "tenacity>=8.1.0,<9.0.0"



  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2


In [5]:
#pip install llama-index-embeddings-huggingface

  Using cached llama_index_embeddings_huggingface-0.6.1-py3-none-any.whl.metadata (458 bytes)
  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
Using cached llama_index_embeddings_huggingface-0.6.1-py3-none-any.whl (8.9 kB)
Using cached sentence_transformers-5.1.2-py3-none-any.whl (488 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
#pip install llama-index-llms-ollama

  Using cached llama_index_llms_ollama-0.9.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached ollama-0.6.0-py3-none-any.whl.metadata (4.3 kB)
Using cached llama_index_llms_ollama-0.9.0-py3-none-any.whl (8.6 kB)
Using cached ollama-0.6.0-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
#pip install llama-index-vector-stores-qdrant


  Using cached llama_index_vector_stores_qdrant-0.8.6-py3-none-any.whl.metadata (476 bytes)
Using cached llama_index_vector_stores_qdrant-0.8.6-py3-none-any.whl (14 kB)


In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import qdrant_client
collection_name="chat_with_docs"
client=qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

In [3]:
from llama_index.core import SimpleDirectoryReader
input_dir_path='./docs'
loader=SimpleDirectoryReader(
    input_dir=input_dir_path,
    required_exts=[".pdf"],
    recursive=True
)
docs=loader.load_data()

2025-11-11 12:45:41,002 - INFO - NumExpr defaulting to 8 threads.


In [4]:
type(docs)

list

In [5]:
len(docs)

32

In [6]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext

def create_index(documents):

    vector_store = QdrantVectorStore(client=client,
                                     collection_name=collection_name)
    
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    index = VectorStoreIndex.from_documents(documents,
                                            storage_context=storage_context)
    
    return index

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                   trust_remote_code=True)

Settings.embed_model = embed_model

index = create_index(docs)

2025-11-11 12:46:06,287 - INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
2025-11-11 12:46:08,367 - INFO - 1 prompt is loaded, with the key: query
2025-11-11 12:46:08,425 - INFO - HTTP Request: GET http://localhost:6333/collections/chat_with_docs/exists "HTTP/1.1 200 OK"
2025-11-11 12:46:08,494 - INFO - HTTP Request: GET http://localhost:6333/collections/chat_with_docs "HTTP/1.1 200 OK"
2025-11-11 12:48:01,707 - INFO - HTTP Request: PUT http://localhost:6333/collections/chat_with_docs/points?wait=true "HTTP/1.1 200 OK"


In [14]:
#pip install -U ipywidgets jupyterlab_widgets

   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 914.9/914.9 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------------------- -------------------- 1.0/2.2 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 5.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab_widgets
    Found existing installation: jupyterlab-widgets 1.0.0
    Uninstalling jupyterlab-widgets-1.0.0:
      Successfully uninstalled jupyterlab-widgets-1.0.0
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.8.1
    Uninstalling ipywidgets-7.8.1:
      Successfully uninstalled ipywidgets-7.8.1
Note: you may need to restart the kernel to use updated pack

In [8]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

# pick one that works from CLI: "llama3.2:1b" or "gemma3:1b"
llm = Ollama(
    model="gemma3:1b",      # or "llama3.2:1b"
    request_timeout=120.0,
)

Settings.llm = llm
print("Using Ollama model:", Settings.llm.model)


Using Ollama model: gemma3:1b


In [9]:
from llama_index.core import PromptTemplate

template = """Context information is below:
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner,
              incase you don't know the answer say 'I don't know!'
            
              Query: {query_str}
        
              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

In [10]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
    top_n=3
)

In [11]:
query_engine = index.as_query_engine(similarity_top_k=10,
                                     node_postprocessors=[rerank],llm=llm)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

response = query_engine.query("What exactly is DSPy?")

2025-11-11 12:48:06,794 - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
2025-11-11 12:48:07,276 - INFO - HTTP Request: POST http://localhost:6333/collections/chat_with_docs/points/search "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-11 12:48:22,855 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [12]:
from IPython.display import Markdown, display

display(Markdown(str(response)))

DSPy is a framework for programming language signatures. It’s used to assign work to language models, allowing them to be prompted in a structured way. It offers benefits like self-improving and pipeline-adaptive prompts, and can be used for tasks like question-answering and multi-hop question answering. It’s a way to define functions and their expected behavior using a concise, declarative style.
